In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [12]:
data = pd.read_csv('churn_modelling_mock.csv')


In [13]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,23882676,Smith,840,France,Female,36,8,55632.06,1,0,0,98692.59,1
1,2,21241085,Jones,419,Spain,Female,21,9,23820.59,3,0,0,100328.10,0
2,3,35705918,Brown,620,Germany,Female,59,4,183841.00,1,0,0,26725.66,0
3,4,71446290,Johnson,424,France,Male,42,7,125001.53,4,1,1,52497.78,1
4,5,90595402,Williams,406,France,Male,37,6,65544.36,4,0,1,40068.71,0


In [14]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,840,France,Female,36,8,55632.06,1,0,0,98692.59,1
1,419,Spain,Female,21,9,23820.59,3,0,0,100328.10,0
2,620,Germany,Female,59,4,183841.00,1,0,0,26725.66,0
3,424,France,Male,42,7,125001.53,4,1,1,52497.78,1
4,406,France,Male,37,6,65544.36,4,0,1,40068.71,0


In [15]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [16]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,840,France,0,36,8,55632.06,1,0,0,98692.59,1
1,419,Spain,0,21,9,23820.59,3,0,0,100328.10,0
2,620,Germany,0,59,4,183841.00,1,0,0,26725.66,0
3,424,France,1,42,7,125001.53,4,1,1,52497.78,1
4,406,France,1,37,6,65544.36,4,0,1,40068.71,0


In [17]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder(sparse_output=False)
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [18]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [19]:
geo_encoder_df=pd.DataFrame(geo_encoder, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [20]:
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
19995,0.0,1.0,0.0
19996,0.0,0.0,1.0
19997,0.0,1.0,0.0
19998,0.0,0.0,1.0


In [21]:
data=pd.concat([data.drop('Geography', axis=1), geo_encoder_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,840,0,36,8,55632.06,1,0,0,98692.59,1,1.0,0.0,0.0
1,419,0,21,9,23820.59,3,0,0,100328.10,0,0.0,0.0,1.0
2,620,0,59,4,183841.00,1,0,0,26725.66,0,0.0,1.0,0.0
3,424,1,42,7,125001.53,4,1,1,52497.78,1,1.0,0.0,0.0
4,406,1,37,6,65544.36,4,0,1,40068.71,0,1.0,0.0,0.0


In [22]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)
    
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [23]:
X = data.drop('Exited', axis=1)
y = data['Exited']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [24]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [25]:
X_train.shape[1]

12

In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [27]:
#Build our model
model = Sequential([
  Dense(64, activation='relu', input_shape=(X_train.shape[1],)), #!st HL
  Dense(32, activation='relu'), #2nd HL
  Dense(1, activation='sigmoid'), #O/P HL
])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001)


In [30]:
#compliling the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
#setup the the tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback= TensorBoard(log_dir=log_dir, histogram_freq=1)

In [32]:
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [65]:
history = model.fit(
  X_train, y_train,
  validation_data=(X_test, y_test),
  epochs=100,
  callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
3/3 [==============================] - 0s 41ms/step - loss: 0.4909 - accuracy: 0.8250 - val_loss: 0.4987 - val_accuracy: 0.7000
Epoch 2/100
3/3 [==============================] - 0s 18ms/step - loss: 0.4868 - accuracy: 0.8250 - val_loss: 0.5011 - val_accuracy: 0.7000
Epoch 3/100
3/3 [==============================] - 0s 17ms/step - loss: 0.4822 - accuracy: 0.8250 - val_loss: 0.5041 - val_accuracy: 0.6500
Epoch 4/100
3/3 [==============================] - 0s 17ms/step - loss: 0.4780 - accuracy: 0.8250 - val_loss: 0.5074 - val_accuracy: 0.6500
Epoch 5/100
3/3 [==============================] - 0s 17ms/step - loss: 0.4720 - accuracy: 0.8250 - val_loss: 0.5067 - val_accuracy: 0.6500
Epoch 6/100
3/3 [==============================] - 0s 17ms/step - loss: 0.4673 - accuracy: 0.8250 - val_loss: 0.5091 - val_accuracy: 0.6500
Epoch 7/100
3/3 [==============================] - 0s 17ms/step - loss: 0.4621 - accuracy: 0.8250 - val_loss: 0.5099 - val_accuracy: 0.6500
Epoch 8/100
3/3 [===

In [33]:
model.save('model.h5')

e:\AIML Practice\ANN Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [68]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [72]:
%tensorboard --logdir logs


Reusing TensorBoard on port 6007 (pid 26944), started 0:00:02 ago. (Use '!kill 26944' to kill it.)